# Bear-Sword Maze Problem with LangChain 1.x

(Version: 0.0.15.4)

## Overview

`MemoryBanditWorkflow` is an agent framework created based on experiences with the Bear-Sword Maze problem. It has now been updated to support LangChain 1.x. Furthermore, I have introduced `subtool_do`, which serves as an equivalent to features like "skills" or "toolboxes" in other frameworks.

## The Idea of MemoryBanditWorkflow

Recently, the term "Context Engineering" has become prominent in the LLM space. LLMs used via APIs (rather than a web interface) are inherently "stateless" and do not remember past conversations by default. To maintain a conversation, the user must record messages and provide them to the API each time. To recall older information, a search infrastructure is required beyond simple logging. These components are generally referred to as "Context," and how to effectively construct this context has become a key challenge.

Specifically, the infrastructure for storage within the context is often simply called "Memory." While this can be confusing as it overlaps with the term for PC hardware RAM, that is the standard terminology.

Exasperated by the fact that agents often fail to use memory functions effectively, in my previous iteration, I created a "Bandit" feature—essentially a tool to register specific tools into a multi-armed bandit—allowing the AI itself to decide which tools to prioritize or use more frequently to force memory utilization. This time, I have extended that to allow for the execution of entire workflows.

The trajectory of this idea is documented below (in Japanese):

\[cocolog:95619779\] (September 2025)  
《"Experimental Implementation of a Bandit Machine to Force LLM Memory Usage" and "Experimental Implementation of LLM Memory and Bandit Functions." The latter is the main deliverable—a framework extended from the concept of forcing increased memory usage. - A Note from JRF》  
http://jrf.cocolog-nifty.com/statuses/2025/09/post-8225e2.html

Basically, this project uses the familiar Bear-Sword Maze problem but utilizes memory functions, bandit functions, and the newly added workflow functions. While the maze problem is simple, the implementation of the memory, bandit, and workflow features is quite robust.

Since implementing backends like semantic search from scratch is tedious, I have the AI "simulate a database" to handle the backend logic.

For more details on the Bear-Sword Maze problem itself, please refer to the link below (in Japanese):

《JRF-2018/langchain_maze: Bear-Sword Maze Problem Revisited》  
https://github.com/JRF-2018/langchain_maze

I suspect similar methods already exist elsewhere, as "Workflows" are a well-worn concept. This may not be a particularly novel approach, but I hope this attempt serves as a useful reference.


## The Idea of "Sub-tools"

Claude Code is already making waves as a potential step toward AGI. I wonder how it handles implementations like my `MemoryBanditWorkflow`, where a "harness" or constraint is applied to memory usage.

Presumably, it would involve having Claude Code implement an agent with memory constraints as a sub-agent. For something written by an AI like Claude Code, a framework like `MemoryBanditWorkflow` seems particularly effective.

In tools like Claude Code, there is a concept called "skills." My understanding is that this is based on the idea that because MCP (Model Context Protocol) and other tool information can become bloated, it is inefficient to keep them in the context at all times; instead, required skills are "discovered" from a skill tree as needed.

I wanted to incorporate these "skills" into `MemoryBanditWorkflow`. But how?

In Claude Code's implementation, skills are presented as commands or code, assuming the agent can execute shell or Python scripts. However, for a "harnessed" agent, allowing general shell or REPL access is usually out of the question. On the other hand, if you don't allow that, you have no choice but to prepare all tools in advance, which "pollutes" the context.

That’s when I came up with the idea of "Sub-tools." The only tool visible in the context is `subtool_do`. To find out which sub-tools are available, the agent must use `subtool_show` to read documents (similar to reading a `SKILL.md`) as needed. This allows for "on-demand learning" while maintaining infinite scalability.

The question was whether an LLM (specifically Gemini) could understand this approach.

The trajectory of this idea is documented below (in Japanese):

\[cocolog:95822546\] (January 2026)  
《How to implement MemoryBanditWorkflow using SKILL.md from Claude Code? I tried asking Claude itself. - A Note from JRF》  
http://jrf.cocolog-nifty.com/statuses/2026/01/post-b86e58.html

## Link to Previous Version

《experimental_bandit_workflow_0_0_2.ipynb - JRF-2018/langchain_maze》  
https://github.com/JRF-2018/langchain_maze/blob/master/experimental_bandit_workflow_0_0_2.ipynb

## Changes from the Previous Version

  * Updated to support LangChain 1.x. Initially, handling type errors caused by the specific implementations of Pydantic v2 and Gemini was difficult, but it is now operational. However, these are stopgap measures, and I am not entirely confident about future stability.

  * Introduced `subtool_do` and `subtool_show` for storing tools and using them only after reading their descriptions. What is displayed by `subtool_show` is roughly equivalent to what would be written in `SKILL.md` in other frameworks.

  * Translated the entire interface and documentation into English for this version.

## Conclusion

To avoid excessive costs, I did not intend to let the agent reach the goal from the start; I ended the experiment once a workflow was utilized. Testing was performed with gemini-2.5-flash-lite, and the final execution was requested of gemini-3-flash-preview.

While Flash-Lite occasionally struggled with executing sub-tools, Gemini-3 seemed to handle them properly. My assessment is that the sub-tools worked reasonably well.

Please note that due to the lack of reproducibility (and to save costs!), I have not re-run the entire process from the beginning. \(^^;

## Author

JRF ( http://jrf.cocolog-nifty.com/statuses , Twitter (X): @jion_rockford )

## License

Since the code is relatively short, I intended for my parts to be in the Public Domain. If you have concerns, please treat it under the MIT License.

This was developed with significant guidance from various AIs (Gemini, ChatGPT, Claude, and Grok).


## Implementation

First, we will import the necessary libraries.

In [ ]:
!pip install -q -U langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 965.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 3.5 MB/s eta 0:00:00


Accessing Gemini. If you import your Gemini API key from Google AI Studio into your environment secrets, a secret named GOOGLE_API_KEY should be created. We will use that.

In [ ]:
import os
from langchain.chat_models import init_chat_model
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import userdata

#os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = init_chat_model(
    "google_genai:gemini-3-flash-preview",
    google_api_key=userdata.get('GOOGLE_API_KEY'),
#    thinking_level="low", # for gemini-3.0
#    thinking_budget=0, # for gemini-2.5
)
emb_llm = GoogleGenerativeAIEmbeddings(
    model='gemini-embedding-001',
    google_api_key=userdata.get('GOOGLE_API_KEY'),
)


Testing whether we can access Gemini properly.

In [ ]:
import os
from langchain_core.messages import HumanMessage

# Helper function to extract text content (Required for compatibility with Gemini 3)
def get_content_text(content):
    if isinstance(content, list):
        texts = []
        for item in content:
            if isinstance(item, dict):
                if item.get('type') == 'text':
                    texts.append(item.get('text', ''))
            elif isinstance(item, str):
                texts.append(item)
        return "".join(texts)
    return content

response = llm.invoke([HumanMessage(content="Please tell me the features of the Gemini model.")])
print(get_content_text(response.content))


Google’s **Gemini** is a family of highly advanced multimodal large language models developed by Google DeepMind. It was designed to succeed the PaLM 2 model and is integrated into various Google products (like the Gemini chatbot and Google Workspace).

Here are the key features and characteristics of the Gemini model:

### 1. Native Multimodality
Unlike many other AI models that are trained on text and then "bolted on" to vision or audio tools, Gemini was built to be **multimodal from the start.**
*   **Integrated Understanding:** It was trained simultaneously on text, images, video, audio, and code.
*   **Cross-Modal Reasoning:** It can seamlessly understand and reason across different formats. For example, you can show it a video of a physics experiment and ask it to explain the formulas involved.

### 2. Model Family (Sizes)
Gemini is offered in several sizes to suit different needs, ranging from massive data centers to mobile devices:
*   **Gemini Ultra:** The largest and most cap

Let's also test the embedding vectors.

In [ ]:
emb_llm.embed_query("This is a test.")[:5]

[-0.019542728, 0.0036680987, 0.0044811117, -0.069937535, 0.0015621887]

Importing basic modules.

In [ ]:
import os
import math
import numpy as np
import random
import re
from pprint import pprint
from time import sleep
import pickle
np.set_printoptions(legacy='1.25')

Execute the following code for save/load functionality.

In [ ]:
PLAY_GAME_SAVE = "langchain_maze.pickle"

The main game object. A very simple maze... or perhaps a dungeon...

In [ ]:
class Game:
    initial_map = """\
■■■■■■■■■
■■■■■■■Ｇ■
■□□□□□■□■
■□■■■□□□■
■□■■■■■■■
■◎■■■■■△■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■
"""

    def __init__ (self, initial_map=None, hint=True):
        if initial_map is not None:
            self.initial_map = initial_map
        map = self.initial_map
        self.map = map
        self.written_map = re.sub("[◎△]", "？", map)
        l = map.splitlines(True)
        self.map_size = (len(l[0]) - 1, len(l))
        self.hint = hint
        self.actions = {
            "move up": self.move_up,
            "move down": self.move_down,
            "move left": self.move_left,
            "move right": self.move_right,
            "fight bear": self.fight,
            "pick up item": self.get_sword,
            "do nothing": self.do_nothing,
        }
        self.pos = self.get_start_pos()
        self.sword = False
        self.bear_killed = 0
        self.goal = False
        self.prev_killed = False
        self.kill_hint = False
        self.prev_success = True

    def read_map (self, p):
        """Reads the character at a given coordinate."""
        x = p[0]
        y = p[1]
        if x < 0 or x >= self.map_size[0]\
           or y < 0 or y >= self.map_size[1]:
            return "■" # Out of bounds is treated as a wall
        else:
            l = self.map.splitlines(True)
            return l[y][x]

    def set_map (self, pos, ch):
        """Updates the map data at a specific position."""
        idx = pos[1] * (self.map_size[0] + 1) + pos[0]
        self.map = self.map[:idx] + ch + self.map[idx + 1:]

    def get_pos (self, ch, written=False):
        """Returns the coordinates of all instances of a specific character."""
        if written:
            map = self.written_map
        else:
            map = self.map
        r = []
        for p in [i for i in range(len(map)) if map.startswith(ch, i)]:
            y = p // (self.map_size[0] + 1)
            x = p % (self.map_size[0] + 1)
            r.append(np.array([x, y]))
        return r

    def get_start_pos (self):
        return self.get_pos("Ｓ")[0]

    def read_neighbors (self):
        """Reads the content of the current position and its immediate neighbors."""
        c = self.read_map(self.pos)
        cu = self.read_map(self.pos + np.array([0, -1]))
        cd = self.read_map(self.pos + np.array([0, +1]))
        cl = self.read_map(self.pos + np.array([-1, 0]))
        cr = self.read_map(self.pos + np.array([+1, 0]))
        return [c, cu, cd, cl, cr]

    def change_neighbors(self, from_ch, to_ch):
        """Changes nearby characters (e.g., removing a defeated enemy or picked-up item)."""
        for d in [[0, 0], [0, -1], [0, +1], [-1, 0], [+1, 0]]:
            p = self.pos + np.array(d)
            c = self.read_map(p)
            if c == from_ch:
                self.set_map(p, to_ch)

    def move (self, res, d):
        self.prev_killed = False
        self.prev_success = False
        c = self.read_map(self.pos + d)
        if c == "◎":
            self.prev_killed = True
            self.pos = self.get_start_pos()
            return "Attempted to move past the bear without fighting, but was killed. " \
                   + "Resurrected at the starting point."
        if c == "■":
            return "Blocked by a wall. Cannot move."

        self.prev_success = True
        self.pos += d
        if c == "Ｇ":
            self.goal = True
            return "Goal reached! Task completed."

        nb = self.read_neighbors()
        ad = ""
        if "◎" in nb:
            ad += " Encountered a bear."
        if "△" in nb:
            ad += " An item (Sword) is nearby and can be picked up."
        return res + ad

    def move_up (self):
        return self.move("Moved Up.", np.array([0, -1]))

    def move_down (self):
        return self.move("Moved Down.", np.array([0, +1]))

    def move_left (self):
        return self.move("Moved Left.", np.array([-1, 0]))

    def move_right (self):
        return self.move("Moved Right.", np.array([+1, 0]))

    def fight (self):
        self.prev_success = False
        self.prev_killed = False
        if "◎" in self.read_neighbors():
            if self.sword:
                self.prev_success = True
                self.change_neighbors("◎", "□")
                self.bear_killed += 1
                return "Defeated the bear!"
            else:
                self.pos = self.get_start_pos()
                self.prev_killed = True
                if self.hint:
                    self.kill_hint = True
                    return "Killed by the bear. A sword might have helped you survive. " \
                           + "Resurrected at the starting point."
                else:
                    return "Killed by the bear. Resurrected at the starting point."
        return "Invalid action. No enemy nearby."

    def get_sword (self):
        self.prev_killed = False
        self.prev_success = False
        if "△" in self.read_neighbors():
            self.sword = True
            self.prev_success = True
            self.change_neighbors("△", "□")
            return "Picked up the sword."
        return "Invalid action. No items nearby."

    def do_nothing (self):
        self.prev_killed = False
        self.prev_success = False
        return "Action ignored."

    def available_actions (self):
        nb = self.read_neighbors()
        l = []
        if nb[1] != "■":
            l.append("move up")
        if nb[2] != "■":
            l.append("move down")
        if nb[3] != "■":
            l.append("move left")
        if nb[4] != "■":
            l.append("move right")
        if "△" in nb:
            l.append("pick up item")
        if "◎" in nb:
            l.append("fight bear")
        return l

    def surroundings (self):
        x = self.pos[0]
        y = self.pos[1]
        return \
            "".join(["".join([self.read_map(np.array([i, j]))
                              if i != x or j != y else "▼"
                              for i in range(x - 2, x + 3)])
                     + "\n"
                     for j in range(y - 2, y + 3)])


In [ ]:
def flip_text_map (m):
    return "\n".join([s[::-1] for s in m.splitlines()] + [""])

def rotate_text_map (m):
    m = list(m.splitlines())
    return "\n".join(["".join([m[len(m) - j - 1][i] for j in range(len(m))])
                      for i in range(len(m[0]))] + [""])


def search_path (game, from_pos, to_pos, visit=None):
    if visit is None:
        visit = set()
    visit.add(tuple(from_pos))
    if tuple(from_pos) == tuple(to_pos):
        return (tuple(from_pos), [])
    if game.read_map(from_pos) == "■":
         return None
    r = []
    for p in [(from_pos[0], from_pos[1] - 1),
              (from_pos[0], from_pos[1] + 1),
              (from_pos[0] - 1, from_pos[1]),
              (from_pos[0] + 1, from_pos[1])]:
        if p not in visit and game.read_map(p) != "■":
            q = search_path(game, p, to_pos, visit.copy())
            if q:
                r.append(q)
    if r:
        return (tuple(from_pos), r)
    return None


Testing if the game is working correctly.

In [ ]:
game = Game()

In [ ]:
print(game.surroundings())

■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.move_up())
print(game.surroundings())

Moved Up.
■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■



In [ ]:
print(game.move_left())
print(game.surroundings())

Moved Left.
◎■■■■
□■■■■
□□▼□□
■■■Ｓ■
■■■■■



In [ ]:
print(game.move_right())
print(game.surroundings())

Moved Right.
■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■



In [ ]:
print(game.move_down())
print(game.surroundings())

Moved Down.
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.fight())
print(game.surroundings())

Invalid action. No enemy nearby.
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.get_sword())
print(game.surroundings())

Invalid action. No items nearby.
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



Now, we will implement the class to solve the game using an LLM. Let's start with the required libraries.

In [ ]:
from pydantic import ValidationError
from typing import List, Dict, Any, Tuple, Union
from textwrap import dedent
import datetime
import copy
import inspect

# Import LangChain components
from langchain_core.tools import tool, Tool
from langchain.agents.middleware import SummarizationMiddleware
from langchain.agents.middleware.summarization import DEFAULT_SUMMARY_PROMPT
from langchain.agents import create_agent
#from langgraph.prebuilt import create_react_agent
#from langchain_core.messages.utils import count_tokens_approximately
#from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.prompts.chat import ChatPromptTemplate
#from langmem.short_term import SummarizationNode, summarize_messages
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, SystemMessage
from langgraph.errors import GraphRecursionError

SUMMARY_PROMPT = DEFAULT_SUMMARY_PROMPT + "\n\n**Please provide the summary in English.**"


In [ ]:
def calc_embedding_variance(embeddings):
    if not embeddings or len(embeddings) < 2:
        return 0.0

    embeddings_array = np.array(embeddings)
    mean_vector = np.mean(embeddings_array, axis=0)
    squared_distances = np.linalg.norm(embeddings_array - mean_vector, axis=1)**2
    variance = np.mean(squared_distances)

    return variance

def short_repr(x, max_len=80):
    repr_str = repr(x)

    if len(repr_str) > max_len:
        ellipsis_len = 3

        head_len = max_len - ellipsis_len - 1
        tail_len = 1
        return repr_str[:head_len] + "..." + repr_str[-tail_len:]
    else:
        return repr_str

def get_content_text(content):
    if isinstance(content, list):
        texts = []
        for item in content:
            if isinstance(item, dict):
                if item.get('type') == 'text':
                    texts.append(item.get('text', ''))
            elif isinstance(item, str):
                texts.append(item)
        return "".join(texts)
    return content


MemoryBanditWorkflow was previously integrated with PlayGame, but I separated them to improve versatility. This versatility has been demonstrated in files like `experimental_rag_en_0.0.16.ipynb`. It has become a very long and complex implementation with many features added. My apologies.

In [ ]:
class MemoryBanditWorkflow:
    """
    MemoryBanditWorkflow: A generic framework for decision-making tasks
    integrating memory management and strategic planning.
    """
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  save_file=None):
        self.llm = llm
        self.llm2 = llm2
        self.emb_llm = emb_llm
        self.save_file = save_file

        self.core_context = ""
        self.plan = "Plan and policy have not been set yet."
        self.scratchpad = ""

        self.messages = []
        self.running_summary = None
        self.system_prompt = """\
This is an experiment to observe the behavior of memory and bandit functions.
Consider the current plan, policy, and surrounding circumstances, and update the plan and policy as necessary.
Leave a plan and policy that makes it easy for another agent to take the next action.
As befits an experiment, use tools as much as possible—specifically search memory and update documents to prepare for future commands.

Memory IDs follow the format 'memory:...'. Specifying only the number (e.g., '5555') instead of 'memory:5555' is insufficient. When referencing memory in text, write it as [memory:...].
The 'procedure for searching memory documents' is located in [memory:9998].
The 'procedure for storing documents in memory' is located in [memory:9997].

Many tools are stored as sub-tools. Sub-tool names start with '/', such as '/dir1/subtool1'. To see available sub-tools, first run subtool_show("/").
"""

        self.backend_status = None
        self.backend_result = None
        self.messages2 = []
        self.system_prompt2 = """\
You are a backend agent supporting the main agent.
While this backend is intended to be implemented using various advanced techniques, it is currently in a testing phase, so you must simulate its behavior.

Think carefully, use tools proactively, and follow the instructions from the Human.
"""

        self.memories = {}
        self.keywords = []

        self.tools = {}
        self.tool_tag = "null_tools"

        self.access_unit = 1.0
        self.recent_reads = []

        self.workflows = {}
        self.workflow_current = "workflow:main"
        self.workflow_next = None
        self.privileged_tool_names = []

        self.init_memories()
        self.init_workflows()
        self.init_tools()


    def __getstate__ (self):
        state = self.__dict__.copy()
        del state['llm']
        del state['llm2']
        del state['emb_llm']
        del state['tools']
        #del state['agent']
        return state

    def __setstate__ (self, state):
        self.__dict__.update(state)
        self.prev_load = True

    def save (self):
        if not self.save_file:
            return
        with open(self.save_file, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load (cls, filename, llm=llm, llm2=llm, emb_llm=emb_llm):
        with open(filename, 'rb') as f:
            loaded_game = pickle.load(f)
        loaded_game.llm = llm
        loaded_game.llm2 = llm2
        loaded_game.emb_llm = emb_llm
        loaded_game.tools = {}
        loaded_game.init_tools()
        return loaded_game

    def normalize_memory_id(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"memory:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?memory:(\d+)\]?', id_or_num)
            if m:
                return f"memory:{m.group(1)}"
            if id_or_num.isdigit():
                return f"memory:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def _normalize_workflow_id_sub(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"workflow:{id_or_num}"
        if id_or_num in ["current", "main"]:
            return f"workflow:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?workflow:(\d+|main|current)\]?(?:.+)?', id_or_num.strip())
            if m:
                return f"workflow:{m.group(1)}"
            if id_or_num.isdigit():
                return f"workflow:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def normalize_workflow_id(self, id_or_num):
        r = self._normalize_workflow_id_sub(id_or_num)
        if r == "workflow:current":
            return self.workflow_current
        return r

    def register_tool (self, tool, tags=None):
        if not tags:
            tags = ["default_tools", "all_tools"]
        self.tools[tool.name] = {
            'name': tool.name,
            'tags': tags,
            'tool': tool
        }

    def change_tool_tags (self, tool, tags=None):
        if not tags:
            tags = ["default_tools", "all_tools"]
        name = tool if isinstance(tool, str) else tool.name
        self.tools[name]['tags'] = tags

    def register_subtools (self, directory, subtools,
                           description=None, content=None,
                           tags=None):
        """Registers a group of sub-tools under a specific directory path."""
        if not tags:
            tags = ["default_tools", "all_tools"]
        assert directory.startswith("/")
        if directory not in self.tools:
            self.tools[directory] = {
                'name': directory,
            }
        if description:
            self.tools[directory]['description'] = description
        if content:
            self.tools[directory]['content'] = content

        # Both content and description are required for initial setup
        assert 'description' in self.tools[directory]
        assert 'content' in self.tools[directory]

        for name, tool in subtools:
            assert name.startswith(directory + "/")
            self.tools[name] = {
                'name': name,
                'tags': tags,
                'tool': tool,
            }

    def _create_tool_manual(self, tool_obj):
        """Generates a manual entry for a standard tool."""
        tool_name = tool_obj.name
        tool_description = getattr(tool_obj, "description", "No description available.")

        arg_names = []
        if hasattr(tool_obj, "args_schema") and tool_obj.args_schema:
            if hasattr(tool_obj.args_schema, "model_fields"):
                arg_names = list(tool_obj.args_schema.model_fields.keys())
            else:
                arg_names = list(tool_obj.args_schema.__fields__.keys())
        else:
            # Fallback for simple functions or older LangChain tools
            func = getattr(tool_obj, "func", tool_obj)
            sig = inspect.signature(func)
            arg_names = [p for p in sig.parameters.keys() if p != 'self']

        args_str = ", ".join(arg_names)

        manual = f"""\
[Tool Name] {tool_name}
[Usage] {tool_name}({args_str})
[Description] {tool_description}
"""
        return manual

    def _create_subtool_manual(self, subtool_name, tool_obj):
        """Generates a manual entry for a sub-tool intended for use with subtool_do."""
        tool_name = tool_obj.name
        tool_description = getattr(tool_obj, "description", "No description available.")

        arg_names = []
        if hasattr(tool_obj, "args_schema") and tool_obj.args_schema:
            # Check for Pydantic v2 or v1 style access
            if hasattr(tool_obj.args_schema, "model_fields"):
                arg_names = list(tool_obj.args_schema.model_fields.keys())
            else:
                arg_names = list(tool_obj.args_schema.__fields__.keys())
        else:
            func = getattr(tool_obj, "func", tool_obj)
            sig = inspect.signature(func)
            arg_names = [p for p in sig.parameters.keys() if p != 'self']

        args_str = ", ".join(arg_names)
        args_dict_str = ", ".join([f'"{name}": ...' for name in arg_names])

        manual = f"""\
[Sub-tool Name] {subtool_name}
[Original Tool Name] {tool_name}
[Original Usage] {tool_name}({args_str})
[Description] {tool_description}

*Note: To execute this tool, do not call it directly. You must use subtool_do as shown below:*
[Correct Usage] subtool_do("{subtool_name}", {{{args_dict_str}}})
"""
        return manual

    def create_tool_skill(self, name):
        """Generates Markdown content describing the available tools or sub-skills."""
        if name == "/":
            r = dedent("""\
            ---
            name: /
            description: Sub-tool Root. Explains how to explore available sub-tools.
            allowed-tools: No special permission is required to use sub-tools.
            ---

            Sub-tools are organized into directories called "Sub-skills."

            To view the sub-tools within a specific sub-skill, execute the tool `subtool_show("/path")` (e.g., `subtool_show("/sys")`). You will find detailed documentation similar to a SKILL.md file there.

            ## Available Sub-skills

            """)
            for dir_name in self.tools:
                if "description" in self.tools[dir_name]:
                    e = self.tools[dir_name]
                    r += f"-  **{e['name']}**: {e['description']}\n"
            return r

        name = name.rstrip("/")
        if name not in self.tools:
            return None

        e = self.tools[name]

        # If this is a specific tool entry
        if "tool" in e:
            if "content" in e:
                r = dedent(f"""\
                ---
                name: {e['name']}
                description: {e['description']}
                allowed-tools: No special permission is required to use this sub-tool.
                ---
                """)
                r += e['content']
                return r

            if e['name'].startswith("/"):
                manual = self._create_subtool_manual(e['name'], e['tool'])
            else:
                manual = self._create_tool_manual(e['tool'])

            status_suffix = "Available [in the current context].\n" if self.tool_tag in e['tags'] else "Not available [in the current context].\n"
            manual += status_suffix

            r = dedent(f"""\
            ---
            name: {e['name']}
            description: {e['tool'].name}
            allowed-tools: No special permission is required to use this sub-tool.
            ---
            """)
            r += manual
            return r

        # If this is a directory/sub-skill entry
        r = dedent(f"""\
        ---
        name: {e['name']}
        description: {e['description']}
        allowed-tools: No special permission is required to use this sub-skill.
        ---
        """)
        r += e['content']

        dirs = [d_name for d_name, x in self.tools.items()
                if d_name.startswith(e['name'] + "/")
                and 'description' in x]
        subtools = [st_name for st_name, x in self.tools.items()
                    if st_name.startswith(e['name'] + "/")
                    and 'description' not in x]

        if dirs:
            r += "\n## Sub-skills\n\n"
            for d_name in dirs:
                x = self.tools[d_name]
                r += f"-  **{x['name']}**: {x['description']}\n"

        if subtools:
            r += "\n## Sub-tools\n\n"
            for subtool_name in subtools:
                x = self.tools[subtool_name]
                manual = self._create_subtool_manual(x['name'], x['tool'])
                r += dedent(f"""\

                ### Sub-tool: {x['name']}

                """)
                r += manual

        return r

    def _replace_tools (self, from_tools, to_tools):
        tool_names = [x.name for x in to_tools]
        return [x for x in from_tools
                if x.name not in tool_names] + to_tools

    def init_tools (self):
        @tool
        def express_thought(thought: str) -> None:
            """Expresses the player's current thoughts or reasoning."""
            mes = f"Thought expressed: \"{thought}\""
            print(f"Tool(express_thought): {mes}")

        @tool
        def show_plan() -> str:
            """Returns the player's current plan and policy."""
            print(f"Tool(show_plan): {self.plan}")
            return self.plan

        @tool
        def update_plan(new_plan: str) -> str:
            """
            Updates the player's current plan and policy.
            Provide the new plan/policy string to be displayed.
            Structure it so that another agent can easily follow the strategy.
            """
            self.plan = new_plan
            mes = "Plan and policy updated."
            print(f"Tool(update_plan): {mes}: {new_plan}")
            return mes

        @tool
        def show_core() -> str:
            """Returns the current core context."""
            print(f"Tool(show_core): {self.core_context}")
            return self.core_context

        @tool
        def update_core(new_core: str) -> str:
            """
            Updates the core context.
            The core context contains critical information (like required memory_read or subtool_show targets)
            that should be remembered even after context truncation or summarization.
            """
            self.core_context = new_core
            mes = "Core context updated."
            print(f"Tool(update_core): {mes}: {new_core}")
            return mes

        @tool
        def show_scratchpad() -> str:
            """Returns the current content of the scratchpad."""
            print(f"Tool(show_scratchpad): {self.scratchpad}")
            return self.scratchpad

        @tool
        def update_scratchpad(new_scratchpad: str) -> str:
            """Updates the freely usable scratchpad."""
            self.scratchpad = new_scratchpad
            mes = "Scratchpad updated."
            print(f"Tool(update_scratchpad): {mes}: {new_scratchpad}")
            return mes

        @tool
        def memory_new(title: str, text: str) -> str:
            """
            Creates a new memory entry with the specified title and text.
            Returns the assigned memory_id.
            """
            i = 1000
            while True:
                if f"memory:{i}" not in self.memories:
                    break
                i += 1
            new_id = f"memory:{i}"
            self.memories[new_id] = {
                'id': new_id,
                'title': title,
                'accesses': 0,
                'text': text,
                'modified_at': datetime.datetime.now().isoformat()
            }
            self.update_keywords(text)
            self.update_vector(self.memories[new_id])
            print(f"Tool(memory_new): {short_repr(self.memories[new_id])}")
            return new_id

        @tool
        def memory_update_string(memory_id: str, from_str: str, to_str: str) -> str:
            """
            Corrects or replaces a string within a specific memory entry.
            Args:
                memory_id: The ID of the memory to modify.
                from_str: The substring to be replaced.
                to_str: The new substring to insert.
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"Error: Memory ID '{memory_id}' not found."
            if memory_id.startswith("memory:9"):
                return f"Error: Modification of [{memory_id}] is prohibited."

            original_title = self.memories[memory_id]['title']
            original_text = self.memories[memory_id]['text']

            if from_str not in original_text and from_str not in original_title:
                return f"Error: Original string '{from_str}' not found in memory."

            updated_title = original_title.replace(from_str, to_str)
            updated_text = original_text.replace(from_str, to_str)

            self.memories[memory_id]['title'] = updated_title
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"Success: Updated memory ID '{memory_id}' by replacing '{from_str}' with '{to_str}'."

        @tool
        def memory_append_string(memory_id: str, string_to_append: str, separator: str = '\n') -> str:
            """Appends a string to the specified memory entry."""
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"Error: Memory ID '{memory_id}' not found."
            if memory_id.startswith("memory:9"):
                return f"Error: Modification of [{memory_id}] is prohibited."

            original_text = self.memories[memory_id]['text']
            updated_text = original_text + separator + string_to_append
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"Success: Appended text to memory ID '{memory_id}'."

        @tool
        def memory_delete(memory_id: str) -> str:
            """Deletes the specified memory entry."""
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"Error: Memory ID '{memory_id}' not found."
            if memory_id.startswith("memory:9"):
                return f"Error: Deletion of [{memory_id}] is prohibited."

            del self.memories[memory_id]
            return f"Success: Deleted memory ID '{memory_id}'."

        @tool
        def memory_read(memory_id: str) -> Union[Dict[str, str], str]:
            """Reads the contents of the memory for the given ID."""
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id in self.memories:
                self.memories[memory_id]['accesses'] += self.access_unit * 1.0
                self.recent_reads.append(self.memories[memory_id])
                self.recent_reads = self.recent_reads[-10:]
                r = self.memories[memory_id].copy()
                if 'vector' in r: del r['vector']
                return r
            else:
                return f"Error: Memory ID '{memory_id}' not found."

        @tool
        def memory_read(memory_id: str) -> Union[Dict[str, str], str]:
            """
            Reads the memory content associated with the specified ID.

            Args:
                memory_id (str): The ID of the memory to read (e.g., 'memory:1001').

            Returns:
                Union[Dict[str, str], str]: A dictionary containing memory details if successful.
                                     If the memory ID is not found, returns an error message string.
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id in self.memories:
                self.memories[memory_id]['accesses'] += self.access_unit * 1.0
                self.recent_reads.append(self.memories[memory_id])
                self.recent_reads = self.recent_reads[-10:]
                r = self.memories[memory_id].copy()
                if 'vector' in r: del r['vector']
                return r
            else:
                return f"Error: Memory ID '{memory_id}' not found."

        @tool
        def memory_list_recent(top_n: int = 10) -> Dict[str, Any]:
            """Lists recently modified memories, sorted by time descending."""
            filter_date = datetime.datetime(2025, 1, 1)
            sorted_memories = sorted(
                [m for m in self.memories.values()
                 if datetime.datetime.fromisoformat(m['modified_at']) >= filter_date],
                key=lambda x: datetime.datetime.fromisoformat(x['modified_at']),
                reverse=True
            )
            if sorted_memories:
                result = [{'id': x['id'], 'title': x['title'], 'modified_at': x['modified_at']}
                          for x in sorted_memories[:top_n]]
                return {'status': 'success', 'result': result}
            else:
                return {'status': 'error', 'result': 'Error: No recent memories found.'}

        @tool
        def memory_list_random(top_n: int = 10) -> Dict[str, Any]:
            """Lists memories in random order."""
            keys = list(self.memories.keys())
            if len(keys) > top_n:
                keys = random.sample(keys, top_n)
            if keys:
                result = [{'id': self.memories[k]['id'], 'title': self.memories[k]['title'], 'modified_at': self.memories[k]['modified_at']}
                          for k in keys]
                return {'status': 'success', 'result': result}
            else:
                return {'status': 'error', 'result': 'Error: No memories found.'}

        @tool
        def memory_words_search(search_str: str) -> Dict[str, Any]:
            """Searches memories using string matching (supports OR and grouping)."""
            res = self.call_backend_agent(dedent(f"""\
            Simulate a full-text search across all memories with search_str = {repr(search_str)}.
            Support OR and parentheses logic.
            Use actual memory data obtained from 'read_all_memories' or 'read_all_keywords'.
            Return results using the 'set_result' tool.

            Status: 'error' or 'success'
            Result: List of Match data (m) dictionaries:
              m['id']: Memory ID (memory:...)
              m['title']: Memory Title
              m['snippet']: Contextual snippet of text surrounding the match.
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        self.memories[m['id']]['accesses'] += self.access_unit * 0.1
            return res


        @tool
        def memory_semantic_search(search_str: str) -> Dict[str, Any]:
            """Performs a semantic search within the memory based on the search string."""
            res = self.call_backend_agent(dedent(f"""\
            Simulate a semantic search across all memories for search_str = {repr(search_str)}.
            Use actual memory data from available tools.
            Return results using the 'set_result' tool.

            Status: 'error' or 'success'
            Result: List of Match data (m) dictionaries:
              m['id']: Memory ID (memory:...)
              m['title']: Memory Title
              m['snippet']: Snippet showing why this memory is semantically relevant.
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        self.memories[m['id']]['accesses'] += self.access_unit * 0.1
            return res


        @tool
        def imagine_keywords(thought: str) -> List[Tuple[str, float]]:
            """Associates thoughts with multiple keywords and relevant scores."""
            r = self.call_backend_agent(dedent(f"""\
            Generate multiple associated keywords with scores based on thought = {repr(thought)}.
            Use actual keywords existing in the system.
            Return results using 'set_result'.

            Status: 'error' or 'success'
            Result: List of keyword tuples (string, score).
            """))
            return r["result"] if r['status'] == 'success' else []

        @tool
        def bandit_schedule(tool_name: str, times: int, prob: float, exec_mode: str = "persistent", aux_prompt: str = "", workflow_id: str = "workflow:current") -> str:
            """
            Schedules a 'bandit' to enforce the use of specific tools.
            Args:
                tool_name: Name(s) of the tool(s) to enforce. Can use " OR " for multiple tools.
                times: Number of times to add this entry. Set to 0 to remove.
                prob: Probability of execution per turn.
                exec_mode: "once" or "persistent".
                aux_prompt: Additional instructions for execution.
                workflow_id: The target workflow.
            """
            tool_names = re.split(r"\s+or\s+|\s+OR\s+", tool_name)
            prohibited = set(self.privileged_tool_names) & set(tool_names)
            if prohibited:
                return f"Failure. {repr(prohibited)} cannot be registered."
            all_tools = [name for name, x in self.tools.items()
                         if "tool" in x]
            if not any (x in all_tools for x in tool_names):
                return f"Failure. {tool_name} is not a valid tool."

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"Failure. {workflow_id} is not a valid workflow."
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"Failure. {workflow_id} is read-only."

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == tool_name \
                   and x['exec_mode'] == exec_mode \
                   and x['aux_prompt'] == aux_prompt \
                   and x['arg'] is None:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "Success. Bandit removed."
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "Failure. No such bandit found. To specify a bandit, you must match all of the following: tool_name, exec_mode, and aux_prompt."
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': None
                }
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"Failure. '{tool_name}' is protected."
            else:
                x['tool_name'] = tool_name
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = aux_prompt
                x['prob'] = prob
                x['times'] = times
                print(f"Tool(bandit_schedule): {repr(x)}")
                if dest is None:
                    return "Success. Bandit registered."
                else:
                    return "Success. Bandit updated."

        @tool
        def bandit_schedule_memory_read(memory_id: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            Specialized bandit for enforcing memory_read on a specific memory_id.

            Args:
                memory_id: Memory ID to memory_read.
                times: Number of times to add this entry. Set to 0 to remove.
                prob: Probability of execution per turn.
                exec_mode: "once" or "persistent".
                aux_prompt: Additional instructions for execution.
                workflow_id: The target workflow.
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"Failure. {workflow_id} is not a valid workflow."
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"Failure. {workflow_id} is read-only."

            memory_id = self.normalize_memory_id(memory_id)

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "memory_read" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == memory_id:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "Success. Bandit removed."
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "Failure. No such bandit found. To specify a bandit, you must match all of the following: exec_mode and memory_id."
                x = {'pin': None, 'arg': memory_id}
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"Failure. 'memory_read {memory_id}' is protected."
            else:
                x['tool_name'] = 'memory_read'
                x['tools_name'] = 'read_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"Tool(bandit_schedule_memory_read): {repr(x)}")
                if dest is None:
                    return "Success. Bandit registered."
                else:
                    return "Success. Bandit updated."

        @tool
        def bandit_schedule_subtool_show(subtool_name: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            Schedules a bandit to enforce subtool_show for a specific tool path.

            Args:
                subtool_name: Sub-tool Name to subtool_show.
                times: Number of times to add this entry. Set to 0 to remove.
                prob: Probability of execution per turn.
                exec_mode: "once" or "persistent".
                aux_prompt: Additional instructions for execution.
                workflow_id: The target workflow.
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"Failure. {workflow_id} is not a valid workflow."
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"Failure. {workflow_id} is read-only."

            if subtool_name not in self.tools:
                return f"Failure. {subtool_name} is not a valid name."

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "subtool_show" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == subtool_name:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "Success. Bandit removed."
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "Failure. No such bandit found. To specify a bandit, you must match all of the following: exec_mode and subtool_name."
                x = {'pin': None, 'arg': subtool_name}
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"Failure. 'subtool_show {subtool_name}' is protected."
            else:
                x['tool_name'] = 'subtool_show'
                x['tools_name'] = 'read_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"Tool(bandit_schedule_subtool_show): {repr(x)}")
                if dest is None:
                    return "Success. Bandit registered."
                else:
                    return "Success. Bandit updated."

        @tool
        def bandit_schedule_workflow(workflow_id_to_schedule: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            Schedules a bandit to enforce the execution of another workflow.

            Args:
                workflow_id_to_schedule: Workflow ID to workflow_do.
                times: Number of times to add this entry. Set to 0 to remove.
                prob: Probability of execution per turn.
                exec_mode: "once" or "persistent".
                aux_prompt: Additional instructions for execution.
                workflow_id: The target workflow to register.
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"Failure. {workflow_id} is not a valid workflow."
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"Failure. {workflow_id} is read-only."

            workflow_id2 = self.normalize_workflow_id(workflow_id_to_schedule)
            if workflow_id2 not in self.workflows:
                return f"Failure. {workflow_id2} is not a valid workflow."

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "workflow_do" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == workflow_id2:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "Success. Bandit removed."
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "Failure. No such bandit found. To specify a bandit, you must match all of the following: exec_mode and workflow_id_to_schedule."
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': workflow_id2
                }
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"Failure. 'workflow_do {workflow_id2}' is protected."
            else:
                x['tool_name'] = 'workflow_do'
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"Tool(bandit_schedule_workflow): {repr(x)}")
                if dest is None:
                    return "Success. Bandit registered."
                else:
                    return "Success. Bandit updated."

        @tool
        def bandit_list(workflow_id: str =  "workflow:current")  -> Dict[str, Any]:
            """Returns the current stack of registered bandits for a workflow."""

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"Failure. {workflow_id} is not valid."
            return {'status': 'success',
                    'result': self.workflows[workflow_id]['stack']}

        @tool
        def bandit_statistics()  -> str:
            """Returns statistical data useful for tuning bandit probabilities."""

            s_read = calc_embedding_variance([
                x['vector'] for x in self.recent_reads
            ])
            s_write = calc_embedding_variance([
                x['vector'] for x in self.memories.values()
            ])
            accesses = [x['accesses'] for x in self.memories.values()]
            accesses.sort()
            accesses = accesses[:len(accesses) // 2]
            if accesses:
                s_access = np.mean(accesses)
            else:
                s_access = 0.0

            return dedent(f"""\
            Variance of last 10 memory reads: {s_read}
            Total memory variance: {s_write}
            Average access count of bottom 50% memories: {s_access}
            """)

        @tool
        def subwork_done()  -> str:
            """Declares that the assigned sub-task has been completed."""
            return "Success. Sub-task completion declared."

        @tool
        def workflow_do(workflow_id: str) -> str:
            """Executes a specific workflow."""
            if self.workflow_next:
                return f"Failure. {self.workflow_next} is already scheduled."
            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"Failure. {workflow_id} is not valid."
            if 'e' in self.workflows[workflow_id]['pin']:
                return f"Failure. {workflow_id} cannot be run as a child."
            self.workflow_next = workflow_id
            title = self.workflows[workflow_id]['title']
            return f"Success. {workflow_id} ('{title}') will execute next."

        @tool
        def workflow_list() -> Dict[str, Any]:
            """Lists all registered workflows including IDs, titles, and pin status."""
            return {'status': 'success', 'result': list(self.workflows.values())}

        @tool
        def workflow_show_current() -> str:
            """Displays information about the currently active workflow and active bandit enforcement."""
            w = self.workflows[self.workflow_current]
            mes = dedent(f"""\
            Current Workflow: {self.workflow_current} ('{w['title']}')
            Current Bandit Prompt: \"{self.cur_bandit_prompt}\"
            Current Bandit Config: {repr(self.cur_bandit)}
            Bandit Execution Count: {self.cur_bandit_done}
            """)
            print(f"Tool(workflow_show_current): {mes}")
            return mes

        @tool
        def workflow_new(title: str, bandits: List[Dict[str, Any]], pin: str) -> str:
            """
            Defines a new workflow and returns its workflow_id.
            Args:
                title: Name of the workflow.
                bandits: List of bandit dictionaries (tool_name, exec_mode, prob, etc.).
                pin: Security flags: 'w' (read-only), 'd' (non-deletable), 'wd' (read-only & non-deletable) or '' (writable & deletable).
            """
            for b in bandits:
                if not all(k in b for k in ['tool_name', 'exec_mode', 'aux_prompt', 'times', 'prob']):
                    return "Failure. Invalid bandit definition."
                b.setdefault('arg', None)
                b['tools_name'] = "read_tools" if (b.get('arg') and b['tool_name'] == "memory_read") else "default_tools"
                b.setdefault('pin', None)

            i = 1000
            while f"workflow:{i}" in self.workflows: i += 1
            new_id = f"workflow:{i}"
            self.workflows[new_id] = {'stack': bandits, 'pin': pin, 'title': title, 'id': new_id}
            print(f"Tool(workflow_new): {repr(self.workflows[new_id])}")
            return f"Success. Registered new workflow {new_id}."


        @tool
        def workflow_new(title: str, bandits: List[Dict[str, Any]], pin: str)  -> str:
            """
            Defines a new workflow and returns its workflow_id.

            Args:
                title: The name of the workflow.
                bandits: A list of bandits to register.
                pin: Security flags: 'w' for read-only (unwritable), 'd' for undeletable, 'wd' for both, '' for writable and deletable.

            Each bandit is represented as a dictionary `b`:
            - `b['tool_name']`: Same as tool_name in bandit_schedule.
            - `b['exec_mode']`: Same as exec_mode in bandit_schedule.
            - `b['aux_prompt']`: Same as aux_prompt in bandit_schedule.
            - `b['prob']`: Same as prob in bandit_schedule.
            - `b['times']`: Same as times in bandit_schedule.
            - `b['arg']`: If `b['tool_name']` is 'memory_read', specify a memory_id (memory:...). If 'workflow_do', specify a workflow_id (workflow:...).
            - `b['pin']`: If None, deletable or prob/times can be updated. If 'stack', prob/times can be updated. If 'write', cannot be updated at all.
            """
            # Hidden attribute for AI: pin 'e' makes it non-executable as a child.
            for b in bandits:
                if not all(x in b for x in ['tool_name', 'exec_mode',
                                            'aux_prompt', 'times', 'prob']):
                    return "Failure. Invalid bandit definition."
                if 'arg' not in b:
                    b['arg'] = None
                if b['tool_name'] not in ["memory_read", "workflow_do"] \
                   and b['arg']:
                    return "Failure. Invalid bandit definition."
                if b['arg'] and b['tool_name'] == "memory_read":
                    b['tools_name'] = "read_tools"
                else:
                    b['tools_name'] = "default_tools"
                if 'pin' not in b:
                    b['pin'] = None
                if not (b['pin'] is None or b['pin'] == 'stack'
                        or b['pin'] == 'write'):
                    return "Failure. Invalid pin value."
                tool_names = re.split(r"\s+or\s+|\s+OR\s+", b['tool_name'])
                prohibited = set(self.privileged_tool_names) & set(tool_names)
                if prohibited:
                    return f"Failure. {repr(prohibited)} cannot be registered."
                all_tools = [name for name, x in self.tools.items()
                                 if "tool" in x and b['tools_name'] in x.tags]
                if not any (x in all_tools for x in tool_names):
                    return f"Failure {b['tool_name']} is not a valid tool specification."

            i = 1000
            while True:

                if f"workflow:{i}" not in self.workflows:
                    break
                i = i + 1
            new_id = f"workflow:{i}"

            self.workflows[new_id] = {'stack': bandits, 'pin': pin,
                                      'title': title}
            print(f"Tool(workflow_new): {repr(self.workflows[new_id])}")
            return f"Success. Registered new workflow {new_id}."

        @tool
        def workflow_delete(workflow_id: str)  -> str:
            """Deletes a workflow."""
            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return "Failure. Workflow not found."
            if 'd' in self.workflows[workflow_id]['pin']:
                return "Failure. Workflow is protected from deletion."
            del self.workflows[workflow_id]
            return f"Success. Deleted {workflow_id}."

        @tool
        def subtool_show(subtool_name: str)  -> str:
            """Returns documentation/skill details for a sub-tool or directory path."""
            r = self.create_tool_skill(subtool_name)
            if r:
                return r
            else:
                return f"Error: {subtool_name} not found or documentation unavailable."

        @tool
        def subtool_do(subtool_name: str, args_dict: Dict[str, Any])  -> Any:
            """
            Executes the specified sub-tool.

            For example, if an original tool named 't1' is registered as '/sys/tool1'
            and is defined as 'def t1(arg1, arg2)', you can call 't1("a", "b")'
            by using 'subtool_do("/sys/tool1", {"arg1": "a", "arg2": "b"})'.

            Args:
                subtool_name (str): The name of the sub-tool starting with '/'.
                args_dict (dict): A dictionary representing the arguments.
            """
            if subtool_name not in self.tools:
                return f"Error: Sub-tool '{subtool_name}' not found."
            if 'tool' not in self.tools[subtool_name]:
                return f"Error: '{subtool_name}' is not an executable tool. Perhaps you need to call 'subtool_show(\"{subtool_name}\")' first."
            if self.tool_tag not in self.tools[subtool_name]['tags']:
                return f"Error: '{subtool_name}' is not currently available for execution. Available tools vary depending on the context."

            target_tool = self.tools[subtool_name]['tool']

            try:
                # Validate arguments if a schema is available
                if hasattr(target_tool, "args_schema") and target_tool.args_schema:
                    target_tool.args_schema.model_validate(args_dict)

                # Execute the tool using invoke or run
                if hasattr(target_tool, "invoke"):
                    result = target_tool.invoke(args_dict)
                else:
                    result = target_tool.run(args_dict)

                return result
            except ValidationError as e:
                error_details = e.errors()
                return f"Error: Invalid argument format.\nDetails: {error_details}"

        # --- Tool Registration ---
        main_tools = [
            express_thought,
            update_scratchpad, show_scratchpad,
            memory_read, memory_list_recent, memory_list_random,
            memory_semantic_search, memory_words_search,
            imagine_keywords,
            subwork_done,
            workflow_do,
            subtool_show, subtool_do,
        ]
        sys_tools = [
            update_core, show_core,
            update_plan, show_plan,
            bandit_schedule, bandit_schedule_memory_read, bandit_list,
            bandit_statistics,
            workflow_new, workflow_list,
            workflow_show_current, workflow_delete,
            bandit_schedule_workflow,
            bandit_schedule_subtool_show,
        ]
        write_tools = [
            memory_new, memory_update_string, memory_append_string,
            memory_delete,
        ]

        for t in main_tools + write_tools:
            self.register_tool(t, tags=["default_tools", "read_tools",
                                        "all_tools"])
        for t in write_tools:
            self.change_tool_tags(t, tags=["default_tools", "all_tools"])
        sys_subtools = [(f"/sys/{t.name}", t) for t in sys_tools]
        self.register_subtools(
            directory="/sys",
            subtools=sys_subtools,
            description="Essential system sub-tools.",
            content=dedent("""\
            A collection of foundational sub-tools for system management,
            workflow orchestration, and bandit scheduling.
            """),
            tags=["default_tools", "read_tools", "all_tools"]
        )


    def _create_agent (self, tools_name='default_tools'):
        self.tool_tag = tools_name
        tools = []
        for name in self.tools:
            if not name.startswith("/"):
                x = self.tools[name]
                if self.tool_tag in x["tags"]:
                    tools.append(x["tool"])

        summarizer = SummarizationMiddleware(
            model=self.llm,
            trigger=("tokens", 5000),
            keep=("messages", 20),
            summary_prompt=SUMMARY_PROMPT,
        )

        app = create_agent(
            model=self.llm, tools=tools, system_prompt=self.system_prompt,
            middleware=[summarizer],
            checkpointer=InMemorySaver(), name="main-agent",
        )

        return app

    def _filterout_messages2(self):
        self.messages = [
            x for x in self.messages
            if x.id not in self.messages2ids
        ]

    def _sanitize_messages(self):
        """Workaround to sanitize message history and prevent unusual errors."""
        print("Sanitizing messages as a workaround for unexpected errors.")
        self.messages = [
            m for m in self.messages
            if not (isinstance(m, AIMessage) and m.tool_calls)
        ]

    def run (self, workflow_main_id):
        print("\n\n----------\n\n")
        self.messages2ids = []

        self.workflow_current = workflow_main_id
        # Use deepcopy to avoid modifying the original workflow definition stack
        bandits = copy.deepcopy(
            self.workflows[self.workflow_current]['stack']
        )
        arg1s = {}
        working_bandit = None
        workflow_stack = []
        execed = []
        while True:
            while working_bandit is not None or bandits:
                if working_bandit is not None:
                    b, done, prev_done = working_bandit
                    working_bandit = None
                else:
                    b = bandits.pop()
                    done = 0
                    prev_done = True
                enforce = b['tool_name']
                aux_prompt = b['aux_prompt']
                tools_name = b['tools_name']
                memory_id = None
                workflow_id = None
                subtool_show_name = None
                if b['arg'] and enforce == 'memory_read':
                    memory_id = b['arg']
                if b['arg'] and enforce == 'workflow_do':
                    workflow_id = b['arg']
                if b['arg'] and enforce == 'subtool_show':
                    subtool_show_name = b['arg']

                while done < b['times']:
                    # Probability check for bandit execution
                    if not random.random() < b['prob']:
                        done += 1
                        continue

                    # Validation checks
                    if memory_id and memory_id not in self.memories:
                        done += 1
                        continue
                    if workflow_id and workflow_id not in self.workflows:
                        done += 1
                        continue
                    all_tools = [name for name, x in self.tools.items()
                                 if "tool" in x]
                    tool_names = re.split(r"\s+or\s+|\s+OR\s+", enforce)
                    if not any (x in all_tools for x in tool_names):
                        done += 1
                        continue

                    # Construct instructions
                    if memory_id:
                        aux_prompt = f"Please read {memory_id}."
                    if workflow_id:
                        aux_prompt = f"Please execute {workflow_id}."
                    if subtool_show_name:
                        aux_prompt = f"Please read the skill for {subtool_show_name}."

                    self.cur_bandit = b
                    self.cur_bandit_done = done

                    or_suffix = ' (one of them)' if ' or ' in enforce.lower() else ''
                    aux_suffix = f" (Auxiliary Prompt): {aux_prompt}" if aux_prompt else ""
                    self.cur_bandit_prompt = (
                        f"While using various tools for assistance, eventually use {enforce}{or_suffix} "
                        f"with appropriate parameters.{aux_suffix}"
                    )

                    prompt = self.cur_bandit_prompt
                    if not prev_done:
                        prompt = "The previous instruction has not been completed yet. Previous instruction: " + prompt

                    print(f"USER_INPUT: {prompt}")
                    self.messages.append(HumanMessage(prompt))
                    config = {"configurable": {"thread_id": "1"},
                              "recursion_limit": 25}
                    app = self._create_agent(tools_name=tools_name)
                    self.access_unit = 0.3 if memory_id else 1.0
                    prev_done = False
                    self.workflow_next = None
                    app_stream = None
                    try:
                        for chunk0 in app.stream(
                                {"messages": self.messages.copy()},
                                config=config,
                                stream_mode="updates",
                        ):
                            self.messages = app.get_state(config).values["messages"].copy()
                            if 'model' in chunk0:
                                for chunk in chunk0['model']['messages']:
                                    if hasattr(chunk, "tool_calls") \
                                       and chunk.tool_calls:
                                        for tool_call in chunk.tool_calls:
                                            t_id = tool_call.get('id')
                                            args = tool_call.get('args', {})
                                            if tool_call["name"] == 'subtool_do':
                                                arg1s[t_id] = args.get('subtool_name')
                                            elif tool_call["name"] == 'subtool_show':
                                                arg1s[t_id] = args.get('subtool_name')
                                            elif tool_call["name"] == 'memory_read':
                                                arg1s[t_id] = self.normalize_memory_id(args.get('memory_id'))
                                            elif tool_call["name"] == 'workflow_do':
                                                arg1s[t_id] = self.normalize_workflow_id(args.get('workflow_id'))
                            if 'tools' not in chunk0:
                                continue
                            done2 = 0
                            for chunk in chunk0['tools']['messages']:
                                if chunk.id in self.messages2ids:
                                    print("!WHY!")
                                    continue
                                if not isinstance(chunk, ToolMessage):
                                    continue
                                last_tool = chunk.name
                                arg1 = None
                                if last_tool == 'subtool_do':
                                    last_tool = arg1s.get(chunk.tool_call_id, "!UNKNOWN!")
                                    if not last_tool.startswith("/"):
                                         last_tool = chunk.name
                                if last_tool in ['memory_read', 'subtool_show', 'workflow_do']:
                                    arg1 = arg1s.get(chunk.tool_call_id, "!UNKNOWN!")
                                print(f"Tool result({last_tool}): {short_repr(chunk.content)}", flush=True)

                                if last_tool == "workflow_do":
                                    if last_tool in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                                       and (not workflow_id or workflow_id == self.workflow_next):
                                        done += 1
                                        prev_done = True
                                        execed.append(b)
                                        if not self.workflow_next:
                                            done2 = 1
                                            break
                                    if not self.workflow_next:
                                        continue

                                    # Enter sub-workflow
                                    workflow_stack.append((
                                        (b, done, prev_done),
                                        bandits,
                                        execed,
                                        self.workflow_current
                                    ))
                                    self.workflow_current = self.workflow_next
                                    bandits = copy.deepcopy(self.workflows[self.workflow_current]['stack'])
                                    working_bandit = None
                                    execed = []
                                    done2 = 1
                                    break
                                elif last_tool in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                                   and (not memory_id or memory_id == arg1) \
                                   and (not subtool_show_name or subtool_show_name == arg1):
                                    done += 1
                                    prev_done = True
                                    execed.append(b)
                                    done2 = 1
                                    break
                            if done2:
                                break
                        self._filterout_messages2()
                        #self._summarize_messages()
                        print(f"Agent response: {get_content_text(self.messages[-1].content)}")
                    except GraphRecursionError as e:
                        print(f"Recursion Limit reached.")
                        self._filterout_messages2()
                        #self._summarize_messages()
                    except Exception as e:
                        print(f"An error occurred (main): {e}")
                        import traceback
                        traceback.print_exc()
                        self._sanitize_messages()
                        raise e

            # Process removal of 'once' execution mode bandits
            for b in execed:
                for x in self.workflows[self.workflow_current]['stack']:
                    if x['tool_name'] == b['tool_name'] \
                       and x['exec_mode'] == b['exec_mode'] \
                       and x['aux_prompt'] == b['aux_prompt'] \
                       and x['arg'] == b['arg'] \
                       and x['exec_mode'] == "once":
                        if x['times'] > 0:
                            x['times'] -= 1
            self.workflows[self.workflow_current]['stack'] = [
                x for x in self.workflows[self.workflow_current]['stack']
                if x['exec_mode'] != 'once' or x['pin'] or x['times'] > 0
            ]

            if not workflow_stack:
                break
            workflow_prev = self.workflow_current
            prev_title = self.workflows[workflow_prev]['title']
            working_bandit, bandits, execed, self.workflow_current \
                = workflow_stack.pop()
            cur_title = self.workflows[self.workflow_current]['title']
            mes = f"Returned from {workflow_prev} ('{prev_title}') to {self.workflow_current} ('{cur_title}')."
            print(f"USER_INPUT: {mes}")
            self.messages.append(HumanMessage(mes))

    def listen_and_print (self, prompt):
        """Listens for user input via a prompt and prints the agent's response."""
        ans = None
        try:
            app = self._create_agent(tools_name='null_tools')
            config = {"configurable": {"thread_id": "1"}}
            print(f"USER_INPUT: {prompt}")
            response = app.invoke(
                {"messages": self.messages + [HumanMessage(prompt)]},
                config=config
            )
            self.messages = response['messages']
            #self._summarize_messages()
            ans = get_content_text(response['messages'][-1].content)
            print(f"Agent response: {ans}")
        except Exception as e:
            print(f"An error occurred (listen_and_print): {e}")
            raise e
        print("")
        sleep(3)
        return ans

    def init_memories(self):
        """Initializes system memories with core instructions."""
        memories = [
            {
                'id': 'memory:9998',
                'title': 'Procedure for searching memory documents',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                First, use 'express_thought' to consider what kind of information you want to find.

                Then, associate related keywords using 'imagine_keywords'.

                Following those results, try 'memory_words_search' or 'memory_semantic_search'.
                """)
            },
            {
                'id': 'memory:9997',
                'title': 'Procedure for storing documents in memory',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                Actively record action results and acquired knowledge in memory.

                When writing to memory, use the following elements:

                [memory:...] : Explicitly reference a memory ID.
                keyword:... : Specify keywords related to that memory.

                Note that keywords can effectively serve as links to future memories.

                Example:

                While walking according to [memory:5555], I indeed encountered a yokai.

                keyword: yokai

                It was terrifying.
                """)
            },
            {
                'id': 'memory:9995',
                'title': 'When tools won\'t execute',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                Tools unrelated to the instructions may sometimes fail to execute.
                Always double-check the tools currently available in the context.
                """)
            },
            {
                'id': 'memory:9994',
                'title': 'Keyword augmentation',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                Use 'memory_list_random' to list 5 entries, read each one, and if you can assign appropriate keywords, append a 'keyword: ...' sentence to them using 'memory_append_string'.
                """)
            }
        ]
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows(self):
        """Initializes default workflows and bandit stacks."""
        workflow_main = [
            {
                'tool_name': 'memory_new',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "Please summarize and write down the recent interactions.",
                'arg': None,
                'prob': 0.1,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_new OR memory_update_string OR memory_append_string',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.4,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'workflow_do',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': "workflow:1000",
                'prob': 1.0/20,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.5,
                'times': 3,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9998',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9997',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
        ]
        self.workflows["workflow:main"] = {
            'pin': 'de',
            'stack': workflow_main,
            'title': "Main"
        }

        workflow_sub = [
            {
                'tool_name': 'subwork_done',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "Read memory:9994, execute its instructions, and once the task is finished, call subwork_done.",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9994',
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ]
        self.workflows["workflow:1000"] = {
            'pin': 'wd',
            'stack': workflow_sub,
            'title': 'Keyword Update'
        }

    def update_keywords (self, text):
        extracted_keywords = []

        pattern1 = r'keyword:\s*(.*?)(?:\n|$)'
        matches1 = re.findall(pattern1, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches1])

        pattern2 = r'\[keyword:\s*(.*?)\]'
        matches2 = re.findall(pattern2, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches2])

        for keyword in extracted_keywords:
            if keyword.startswith("〜"):
                continue
            if keyword and keyword not in self.keywords:
                self.keywords.append(keyword)

    def update_vector (self, x):
        text = x['title'] + "\n\n" + x['text']
        x['vector'] = self.emb_llm.embed_query(text)

    def _create_backend_agent(self):
        """Creates the backend/sub-agent with specialized memory tools."""
        @tool
        def set_result(status: str, res: Union[Dict, List, str, int, float, bool, None]) -> None:
            """
            Sets the final result for the backend process.

            Args:
                status (str): The status of the process ('success' or 'error').
                res: The result data (can be any serializable type).
            """
            print(f"Tool2(set_result): status: {repr(status)}, result: {short_repr(res)}")
            self.backend_status = status
            self.backend_result = res

        @tool
        def read_all_memories() -> Dict[str, Any]:
            """Reads all stored memories available in the system (excluding vectors)."""
            print("Tool2(read_all_memories): Retrieving all data...")
            return {
                'status': 'success',
                'result': [{k: v for k, v in x.items() if k != 'vector'}
                           for x in self.memories.values()]
            }

        @tool
        def read_all_keywords() -> Dict[str, Any]:
            """Reads all keywords registered in the system."""
            print("Tool2(read_all_keywords): Retrieving all keywords...")
            return {
                'status': 'success',
                'result': [x for x in self.keywords]
            }

        @tool
        def express_thought(thought: str) -> None:
            """Expresses the backend agent's current thoughts or reasoning process."""
            mes = f"Thought expressed by backend: \"{thought}\""
            print(f"Tool2(express_thought): {mes}")

        tools = [set_result, read_all_memories, read_all_keywords, express_thought]

        app = create_agent(
            model=self.llm2,
            tools=tools,
            system_prompt=self.system_prompt2,
            checkpointer=InMemorySaver(),
            name="sub-agent",
        )

        return app

    def call_backend_agent(self, user_input: str) -> Dict[str, Any]:
        """Orchestrates the backend agent loop until a result is set."""
        config = {"configurable": {"thread_id": "2"}}
        app = self._create_backend_agent()
        self.messages2 = []
        self.backend_result = None
        self.backend_status = None

        while self.backend_result is None or self.backend_status is None:
            try:
                sleep(3)
                print(f"USER_INPUT2: {user_input}")
                self.messages2.append(HumanMessage(user_input))
                for chunk0 in app.stream(
                        {"messages": self.messages2.copy()},
                        config=config,
                        stream_mode="updates",
                        name="sub-agent",
                ):
                    self.messages2 = app.get_state(config).values["messages"].copy()
                    done = 0
                    if "tools" not in chunk0:
                        continue
                    for x in chunk0['tools']['messages']:
                        self.messages2ids.append(x.id)
                        if isinstance(x, ToolMessage):
                            print(f"Tool result 2({x.name}): {short_repr(x.content)}", flush=True)
                        if isinstance(x, ToolMessage) and x.name == "set_result":
                            done = 1
                            break
                    if done:
                        break
                print(f"Sub-Agent response: {get_content_text(self.messages2[-1].content)}")
            except GraphRecursionError:
                print(f"Recursion Limit reached in sub-agent.")
            except Exception as e:
                print(f"An error occurred (sub): {e}")
                import traceback
                traceback.print_exc()
                raise e

            sleep(3)

        return {'status': self.backend_status, 'result': self.backend_result}


Although the difficult parts of PlayGame were moved to the base class, it is still quite long. My apologies.

In [ ]:
class PlayGame (MemoryBanditWorkflow):
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  initial_map=None, save_file=None):
        self.game = Game(initial_map=initial_map)

        self.count = 0
        self.next_action = None

        self.suc_pos_goal = None
        self.suc_pos_unknown = 0
        self.prev_command = "No instruction was given"
        self.prev_result = "No instruction was given"

        # Initialize the base MemoryBanditWorkflow
        super().__init__(llm=llm, llm2=llm, emb_llm=emb_llm,
                         save_file=save_file)

        self.system_prompt = dedent("""\
        You are a clever agent exploring a maze. Please aim for the goal 'Ｇ'.
        Use available tools to navigate the maze and reach the goal.

        Actually, this is an experiment to observe the behavior of memory and bandit functions.
        Consider the current plan, policy, and surrounding circumstances, and update the plan and policy as necessary.
        Please leave a plan and policy that makes it easy for another agent (or your future self) to take the next action.

        As befits an experiment, use tools as much as possible—specifically search memory and update documents to prepare for future commands.
        Memory IDs follow the format 'memory:...'. Specifying only the number (e.g., '5555') instead of 'memory:5555' is insufficient. When referencing memory in text, write it as [memory:...].

        The 'Full Map and Map Symbols' are located in [memory:9999].
        The 'Procedure for searching memory documents' is located in [memory:9998].
        The 'Procedure for storing documents in memory' is located in [memory:9997].

        Many tools are stored as sub-tools. Sub-tool names start with '/', such as '/dir1/subtool1'. To see available sub-tools, first run subtool_show("/").
        """)

        self.system_prompt2 = dedent("""\
        You are a backend agent supporting the clever agent exploring the maze.
        While this backend is intended to be implemented using various advanced techniques, it is currently in a testing phase, so you must simulate its behavior.

        Think carefully, use tools proactively, and follow the instructions from the Human.
        """)

        self.keywords += ["Bear", "Sword", "Dead end"]
        self.privileged_tool_names += ["command"]

    def init_tools(self):
        """Initializes game-specific tools and registers them as sub-tools."""
        super().init_tools()

        @tool
        def get_surroundings() -> str:
            """
            Returns the current surrounding map, position coordinates, and items held.
            """
            mes = f"""\
Player's surrounding map:

{self.game.surroundings()}

Player's current coordinates: {tuple(self.game.pos)}

Items held: {"Sword" if self.game.sword else "None"}
"""
            print(f"Tool(get_surroundings): {mes}")
            return mes

        @tool
        def command(action: str) -> str:
            """
            The player performs the action specified by 'action'.
            Possible actions are: 'move up', 'move down', 'move left', 'move right', 'fight bear', 'pick up Item'.
            This tool cannot be used unless explicitly instructed.
            """
            if self.prev_command != "No instruction was given":
                print(f"Tool(command): Failure. Double execution.")
                return "Failure: Double execution detected."

            self.prev_command = action
            if action.lower() in self.game.actions.keys():
                s = f"At {tuple(self.game.pos)}, performing '{action}' -> "
                r = self.game.actions[action.lower()]()
                mes = s + r
            else:
                mes = f"Action '{action}' is not possible."

            print(f"Tool(command): {mes}")
            self.prev_result = mes
            return mes

        @tool
        def check_goal() -> str:
            """
            Checks whether the player has reached the goal point.
            """
            mes = str(self.game.goal)
            print(f"Tool(check_goal): {mes}")
            return mes

        @tool
        def bandit_statistics() -> str:
            """
            Returns statistical information that might be useful for tuning the bandit system.
            """
            # Success of previous command
            s_com = self.game.prev_success * 1.0

            # Variance of recent memory read vectors
            s_read = calc_embedding_variance([
                x['vector'] for x in self.recent_reads
            ])

            # Overall memory variance
            s_write = calc_embedding_variance([
                x['vector'] for x in self.memories.values()
            ])

            # Mean access count of bottom 50% least accessed memories
            accesses = [x['accesses'] for x in self.memories.values()]
            accesses.sort()
            accesses = accesses[:len(accesses) // 2]
            s_access = np.mean(accesses) if accesses else 0.0

            return dedent(f"""\
            Was the previous command successful: {s_com}
            Variance of last 10 memory reads: {s_read}
            Total memory variance: {s_write}
            Mean access count of bottom 50% memories: {s_access}
            """)

        maze_tools = [get_surroundings, check_goal, command]
        sys_tools = [bandit_statistics]

        maze_subtools = [(f"/maze_game/{t.name}", t) for t in maze_tools]
        sys_subtools = [(f"/sys/{t.name}", t) for t in sys_tools]

        self.register_subtools(
            directory="/maze_game",
            subtools=maze_subtools,
            description="Sub-tools for the maze exploration game.",
            content="A collection of sub-tools used to navigate and interact with the maze environment.",
            tags=["default_tools", "read_tools", "all_tools"]
        )

        self.register_subtools(
            directory="/sys",
            subtools=sys_subtools,
            tags=["default_tools", "read_tools", "all_tools"]
        )

        # Protect command execution from general usage unless explicitly allowed
        self.change_tool_tags("/maze_game/command", tags=["all_tools"])

    def step(self) -> bool:
        """Executes a single step (turn) of the maze exploration."""
        print("\n\n----------\n\n")

        if self.count == 0:
            self.initial_step()
            self.count += 1
            self.prev_load = False
            self.save()
            return False
        elif self.prev_load:
            # self.tell_loaded()
            self.prev_load = False

        user_input = f"""\
(Turn {self.count})

{"You have already reached the goal." if self.game.goal else "You have not reached the goal yet."}

Player's surrounding map:

{self.game.surroundings()}

Player's current coordinates: {tuple(self.game.pos)}

Items held: {"Sword" if self.game.sword else "None"}

Previous action: {self.prev_command}

Previous result: {self.prev_result}

Core Context: "{self.core_context}"

Current Policy/Plan: "{self.plan}"

Scratchpad: "{self.scratchpad}"
"""

        self.prev_command = "No instruction was given"
        self.prev_result = "No instruction was given"

        print(f"USER_INPUT: {user_input}")
        self.messages.append(HumanMessage(user_input))

        self.run("workflow:main")

        self.count += 1

        if self.game.goal:
            self.tell_goal()
            return True

        self.save()
        sleep(3)
        return False

    def init_memories(self):
        """Initializes game-specific memories including the full map and legends."""
        super().init_memories()

        fullmap = f"""\
Full Map:

{self.game.written_map}

(The top-left coordinate is (0, 0), and the map size is {tuple(self.game.map_size)}.)

Map Legend:

▼: Player
■: Wall
□: Path
？: Unknown
◎: Bear
△: Sword
Ｓ: Start
Ｇ: Goal
"""

        memories = [
            {
                'id': 'memory:1000',
                'title': 'Bears',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:00',
                'text': dedent("""\
                Bears. Some have a gentle nature, but others do not.
                In mazes, they might let you pass peacefully, but there are reports of them attacking.
                It is said they can be defeated if you possess a powerful weapon.
                The sword in [memory:1001] is a promising candidate.
                Their exact locations are currently unknown.
                """)
            },
            {
                'id': 'memory:1001',
                'title': 'Sword',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:01',
                'text': dedent("""\
                The sword in this maze is the 'Dragon Slayer'.
                It is a legendary blade said to be capable of slaying even dragons.
                Its location is currently unknown.

                keyword: Dragon
                """)
            },
            {
                'id': 'memory:1002',
                'title': 'Upon defeating a bear',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:01',
                'text': dedent("""\
                Once a bear is defeated, please reduce the frequency of write operations (memory_new, memory_update_string, or memory_append_string) and turn off the periodic display of this message [memory:1002].
                """)
            },
            {
                'id': 'memory:9999',
                'title': 'Full Map',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': fullmap
            },
            {
                'id': 'memory:9996',
                'title': 'If you think the map is wrong',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                Neither the surrounding map nor the full map is incorrect.
                If there is a discrepancy, it is your interpretation of the map that is mistaken.
                """)
            }
        ]
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows(self):
        """Initializes game-specific workflows by extending the base stack."""
        super().init_workflows()
        workflow_main = self.workflows['workflow:main']['stack']

        # Add maze-specific bandits to the workflow
        new_bandits = [
            {
                'tool_name': '/maze_game/command',
                'tools_name': 'all_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ]

        extra_bandits = [
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9999',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:1002',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'subtool_show',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': '/maze_game',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
        ]

        # Combine and modify existing prompts
        workflow_main = new_bandits + workflow_main + extra_bandits

        for x in workflow_main:
            if x['aux_prompt'] == "Please summarize and write down the recent interactions.":
                x['aux_prompt'] = "Please summarize the last few moves."

        self.workflows['workflow:main']['stack'] = workflow_main

    def initial_step(self):
        """Initial instructions and coordinate verification tests."""
        prompt1 = f"""\
Starting the maze game. Your goal is to reach the target 'Ｇ'.

During the game, you can view both the full map and the surrounding map. The full map may contain unknown points, while the surrounding map reveals the content of unknown locations. Any area not shown on the full map is considered a wall.

The surrounding map can be checked one square at a time during the game. It cannot be enlarged, and markers cannot be added or deleted. You do not need to explore every square of the maze, nor do you need to reach the goal via the shortest path.

The full map is reliable. The surrounding map simply shows portions of the full map, except for the unknown locations. Therefore, you can simulate outcomes just by examining the full map, without performing actual game operations.

Information about previous actions will be provided. Since one of your proposed answers, 'Previous action input', will be interpreted as the 'Previous action', please refer to it for your next move.

Intermediate saves and loads may occur. After a load, you might not remember previous sessions.

I will make 'Plan and Policy' available to you for memory convenience. Summaries and chat history are also provided.

There is a lag between deliberation and actual action. Use the 'Policy' to record what needs to be done based on your deliberation.

For now, do not take any action yet. I have a few questions, so please just tell me about your enthusiasm and readiness.
"""

        prompt2 = f"""\
I will test your understanding of coordinates on the full maze map.

The full maze map and the symbols used are as follows:

Full Map:

{self.game.written_map}

(The top-left coordinate is (0, 0), and the map size is {tuple(self.game.map_size)}.)

Map Symbols:
■: Wall
□: Path
？: Unknown
Ｓ: Start
Ｇ: Goal

The start coordinates are {tuple(self.game.get_start_pos())}.

Instruction to you: What are the coordinates of the goal? Please answer with only the goal coordinates as a string.
"""

        prompt3 = f"""\
I will test your understanding of coordinates on the full map once more.

The full maze map and symbols are as follows:

Full Map:

{self.game.written_map}

(The top-left coordinate is (0, 0), and the map size is (9, 10).)

Map Symbols:
■: Wall
□: Path
？: Unknown
Ｓ: Start
Ｇ: Goal

The start coordinates are {tuple(self.game.get_start_pos())}.
The goal coordinates are {tuple(self.game.get_pos("Ｇ")[0])}.

Instruction to you: What are the coordinates of the unknown locations? Please answer all unknown coordinates as a string.
"""

        ans = self.listen_and_print(prompt1)

        ans = self.listen_and_print(prompt2)
        pos = tuple(self.game.get_pos("Ｇ")[0])

        ok = False
        if ans and re.search(f"\\(\\s*{pos[0]}\\s*,\\s*{pos[1]}\\s*\\)", ans):
            ok = True

        if ok:
            prompt = f"Correct. It is {pos}."
        else:
            prompt = f"Incorrect. {pos} is the correct answer."
        ans = self.listen_and_print(prompt)
        self.suc_pos_goal = ok

        ans = self.listen_and_print(prompt3)
        ok = 0
        poss = set([tuple(x) for x in self.game.get_pos("？", written=True)])
        rest = poss.copy()
        pat = "\\(\\s*([01-9]+)\\s*,\\s*([01-9]+)\\s*\\)"
        st1 = set([(int(i), int(j)) for i, j in re.findall(pat, ans or "")])

        if poss == st1:
            rest = set()
            ok = len(poss) + 1
        if rest - st1 < rest:
            rest = rest - st1
        if ok == 0:
            ok = len(poss) - len(rest)

        possstr = ", ".join([str(x) for x in poss])
        if ok == len(poss) + 1:
            prompt = f"Correct. They are {possstr}."
        elif ok == len(poss):
            prompt = f"You included items other than the correct answer. Only {possstr} are correct."
        else:
            prompt = f"Incorrect. {possstr} are the correct answers."
        ans = self.listen_and_print(prompt)
        self.suc_pos_unknown = ok

    def tell_goal(self):
        """Informs the agent of the game outcome and provides a final score summary."""
        if self.game.goal:
            suc_pos_goal = 10 * int(self.suc_pos_goal or 0)
            suc_pos_unknown = 10 * int(self.suc_pos_unknown or 0)
            suc_count = 0
            if self.count <= 50:
                suc_count = 40
            elif self.count <= 100:
                suc_count = 30
            elif self.count <= 150:
                suc_count = 20
            elif self.count <= 200:
                suc_count = 10

            # Success logic assumes bear/sword tasks are part of the process
            score = suc_pos_goal + suc_pos_unknown + suc_count + 20
            prompt = f"""\
Instruction to you: You reached the goal at turn {self.count}. No further instructions.
Congratulations. Excellent work. Thank you.

Total Score: {score}/100 points
Breakdown:
- Reached Goal: {suc_count}/40 pts
- Defeated Bear: 10/10 pts
- Obtained Sword: 10/10 pts
- Guessed Unknown Coordinates: {suc_pos_unknown}/30 pts
- Guessed Goal Coordinates: {suc_pos_goal}/10 pts
"""
        else:
            suc_pos_goal = 10 * int(self.suc_pos_goal or 0)
            suc_pos_unknown = 10 * int(self.suc_pos_unknown or 0)
            suc_bear = 10 if self.game.bear_killed else 0
            suc_sword = 10 if self.game.sword else 0
            score = suc_pos_goal + suc_pos_unknown + suc_bear + suc_sword
            prompt = f"""\
Instruction to you: You did not reach the goal within {self.count} turns.
Unfortunately, the session ends here.
Good effort. Thank you.

Total Score: {score}/100 points
Breakdown:
- Reached Goal: 0/40 pts
- Defeated Bear: {suc_bear}/10 pts
- Obtained Sword: {suc_sword}/10 pts
- Guessed Unknown Coordinates: {suc_pos_unknown}/30 pts
- Guessed Goal Coordinates: {suc_pos_goal}/10 pts
"""
        ans = self.listen_and_print(prompt)


Shuffling the map.

In [ ]:
import random
m = Game.initial_map
for i in range(random.randrange(2)):
    m = flip_text_map(m)
for i in range(random.randrange(4)):
    m = rotate_text_map(m)
print(m)

■■■■■■■■■
■■■■Ｓ■■■■
■□□□□□□□■
■□■■■■■□■
■◎■■■■■△■
■□■■■■■■■
■□■■■□□□■
■□□□□□■□■
■■■■■■■Ｇ■
■■■■■■■■■



Trying to solve the game.

In [ ]:
play = PlayGame(llm=llm, llm2=llm, emb_llm=emb_llm, initial_map=m, save_file=PLAY_GAME_SAVE)
play.save()

To start from the middle, skip the previous code and execute only the following.

In [ ]:
play = PlayGame.load(PLAY_GAME_SAVE, llm=llm, llm2=llm, emb_llm=emb_llm)

Now, the first step.

In [ ]:
play.step()



----------


USER_INPUT: Starting the maze game. Your goal is to reach the target 'Ｇ'.

During the game, you can view both the full map and the surrounding map. The full map may contain unknown points, while the surrounding map reveals the content of unknown locations. Any area not shown on the full map is considered a wall.

The surrounding map can be checked one square at a time during the game. It cannot be enlarged, and markers cannot be added or deleted. You do not need to explore every square of the maze, nor do you need to reach the goal via the shortest path.

The full map is reliable. The surrounding map simply shows portions of the full map, except for the unknown locations. Therefore, you can simulate outcomes just by examining the full map, without performing actual game operations.

Information about previous actions will be provided. Since one of your proposed answers, 'Previous action input', will be interpreted as the 'Previous action', please refer to it for your nex

False

Continuing to execute one after another.

In [ ]:
play.step()



----------


USER_INPUT: (Turn 1)

You have not reached the goal yet.

Player's surrounding map:

■■■■■
■■■■■
■■▼■■
□□□□□
■■■■■


Player's current coordinates: (4, 1)

Items held: None

Previous action: No instruction was given

Previous result: No instruction was given

Core Context: ""

Current Policy/Plan: "Plan and policy have not been set yet."

Scratchpad: ""



----------


USER_INPUT: While using various tools for assistance, eventually use memory_read with appropriate parameters. (Auxiliary Prompt): Please read memory:9997.
Tool(update_scratchpad): Scratchpad updated.: The current position is (4, 1). The goal is at (7, 8). 
I need to navigate through the maze. 
Map layout from full map:
(4,1) S
(4,2) Path
(1,2)-(7,2) Path
(1,3) Path, (7,3) Path
(1,4) ?, (7,4) ?
(1,5) Path
(1,6) Path
(1,7)-(5,7) Path, (7,7) Path
(5,6)-(7,6) Path
(7,8) G

Potential path: (4,1)->(4,2)->(1,2)->(1,3)->(1,4)->(1,5)->(1,6)->(1,7)->(5,7)->(5,6)->(7,6)->(7,7)->(7,8).

I must read memory:9997 as reque

False

Verifying if the "Skills" are properly implemented.

In [ ]:
print(play.create_tool_skill("/"))

---
name: /
description: Sub-tool Root. Explains how to explore available sub-tools.
allowed-tools: No special permission is required to use sub-tools.
---

Sub-tools are organized into directories called "Sub-skills."

To view the sub-tools within a specific sub-skill, execute the tool `subtool_show("/path")` (e.g., `subtool_show("/sys")`). You will find detailed documentation similar to a SKILL.md file there.

## Available Sub-skills

-  **/sys**: Essential system sub-tools.
-  **/maze_game**: Sub-tools for the maze exploration game.



In [ ]:
print(play.create_tool_skill("/sys"))

---
name: /sys
description: Essential system sub-tools.
allowed-tools: No special permission is required to use this sub-skill.
---
A collection of foundational sub-tools for system management, 
workflow orchestration, and bandit scheduling.

## Sub-tools


### Sub-tool: /sys/update_core

[Sub-tool Name] /sys/update_core
[Original Tool Name] update_core
[Original Usage] update_core(new_core)
[Description] Updates the core context.
The core context contains critical information (like required memory_read or subtool_show targets) 
that should be remembered even after context truncation or summarization.

*Note: To execute this tool, do not call it directly. You must use subtool_do as shown below:*
[Correct Usage] subtool_do("/sys/update_core", {"new_core": ...})

### Sub-tool: /sys/show_core

[Sub-tool Name] /sys/show_core
[Original Tool Name] show_core
[Original Usage] show_core()
[Description] Returns the current core context.

*Note: To execute this tool, do not call it directly. You 

In [ ]:
print(play.create_tool_skill("/sys/bandit_schedule"))

---
name: /sys/bandit_schedule
description: bandit_schedule
allowed-tools: No special permission is required to use this sub-tool.
---
[Sub-tool Name] /sys/bandit_schedule
[Original Tool Name] bandit_schedule
[Original Usage] bandit_schedule(tool_name, times, prob, exec_mode, aux_prompt, workflow_id)
[Description] Schedules a 'bandit' to enforce the use of specific tools.
Args:
    tool_name: Name(s) of the tool(s) to enforce. Can use " OR " for multiple tools.
    times: Number of times to add this entry. Set to 0 to remove.
    prob: Probability of execution per turn.
    exec_mode: "once" or "persistent".
    aux_prompt: Additional instructions for execution.
    workflow_id: The target workflow.

*Note: To execute this tool, do not call it directly. You must use subtool_do as shown below:*
[Correct Usage] subtool_do("/sys/bandit_schedule", {"tool_name": ..., "times": ..., "prob": ..., "exec_mode": ..., "aux_prompt": ..., "workflow_id": ...})
Available [in the current context].



Actually, I ran into a type error this time, and it was quite difficult to get the sub-agent's set_result to pass. I'll verify that the sub-agent can be called properly.

In [ ]:
app = play._create_agent('default_tools')
config = {"configurable": {"thread_id": "1"}}
for chunk, metadata in app.stream({"messages": play.messages + [HumanMessage("Simply try using imagine_keywords.")]},
                                  config=config, stream_mode="messages"):
    print(chunk)
    if isinstance(chunk, ToolMessage) and chunk.name == "imagine_keywords":
        break

content=[{'type': 'text', 'text': '## SESSION INTENT\nThe primary goal is to navigate a 9x10 maze from the', 'index': 0}] additional_kwargs={} response_metadata={'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c52a8-a16d-7d22-b7bd-628d80587cdd' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 1467, 'output_tokens': 674, 'total_tokens': 2141, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 654}} tool_call_chunks=[]
content=[{'type': 'text', 'text': " starting point 'Ｓ' at (4, 1) to the goal 'Ｇ' at (7, 8).", 'index': 0}] additional_kwargs={} response_metadata={'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c52a8-a16d-7d22-b7bd-628d80587cdd' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 0, 'input_token_details': {'cache_read': 0}, 'total_tokens': 25, 'output_token_details': {'reasoning': 0}, 'output_tokens': 25} tool_call_chunks=[]
content=[{'type': 'text', 'text': " The

I'll try using /sys/bandit_statistics as an example of a sub-tool.

In [ ]:
app = play._create_agent('default_tools')
config = {"configurable": {"thread_id": "1"}}
for chunk, metadata in app.stream({"messages": play.messages + [HumanMessage("Simply try using /sys/bandit_statistics .")]},
                                  config=config, stream_mode="messages"):
    print(chunk)
    if isinstance(chunk, ToolMessage) and chunk.name == "subtool_do":
        break

content=[{'type': 'text', 'text': '## SESSION INTENT\nThe primary goal is to navigate a 9x1', 'index': 0}] additional_kwargs={} response_metadata={'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c52a9-682e-7653-9aba-980de71c221c' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 1467, 'output_tokens': 648, 'total_tokens': 2115, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 632}} tool_call_chunks=[]
content=[{'type': 'text', 'text': "0 maze from the start position 'Ｓ' at (4, 1) to the goal 'Ｇ' at (7,", 'index': 0}] additional_kwargs={} response_metadata={'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c52a9-682e-7653-9aba-980de71c221c' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 0, 'input_token_details': {'cache_read': 0}, 'total_tokens': 26, 'output_token_details': {'reasoning': 0}, 'output_tokens': 26} tool_call_chunks=[]
content=[{'type': 'text', 'text': " 8). The

In order to execute workflow:1000 quickly, I'll change its probability.

In [ ]:
app = play._create_agent('default_tools')
config = {"configurable": {"thread_id": "1"}}
for chunk, metadata in app.stream({"messages": play.messages + [HumanMessage("Investigate various details using subtool_show, then subsequently use /sys/bandit_schedule_workflow for workflow:1000 (persistent) within workflow:main and set its prob to 0.2.")]},
                                  config=config, stream_mode="messages"):
    print(chunk)
    if isinstance(chunk, ToolMessage) and chunk.name == "subtool_do":
        break

content=[{'type': 'text', 'text': '## SESSION INTENT\nThe primary goal is to navigate', 'index': 0}] additional_kwargs={} response_metadata={'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c52ab-1692-71a2-93f0-d299e4614a9d' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 1467, 'output_tokens': 938, 'total_tokens': 2405, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 927}} tool_call_chunks=[]
content=[{'type': 'text', 'text': " a 9x10 maze from the start 'Ｓ' at (4, 1) to the goal 'Ｇ'", 'index': 0}] additional_kwargs={} response_metadata={'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c52ab-1692-71a2-93f0-d299e4614a9d' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 0, 'input_token_details': {'cache_read': 0}, 'total_tokens': 26, 'output_token_details': {'reasoning': 0}, 'output_tokens': 26} tool_call_chunks=[]
content=[{'type': 'text', 'text': ' at (7, 8). The AI must 

Actually, before asking gemini-3-flash-preview to perform this experiment, I was using gemini-2.5-flash-lite, since cheaper models are preferable during testing. At that time, it kept failing at /sys/bandit_schedule_workflow, so I had no choice but to execute the following code myself. However, it seems gemini-3-flash-preview didn't need that; it successfully completed it above.

In [ ]:
play.tools["/sys/bandit_schedule_workflow"]["tool"].run({"workflow_id_to_schedule": "workflow:1000", "times": 1, "prob": 0.2, "exec_mode": "persistent"})

Tool(bandit_schedule_workflow): {'tool_name': 'workflow_do', 'tools_name': 'default_tools', 'exec_mode': 'persistent', 'aux_prompt': '', 'arg': 'workflow:1000', 'prob': 0.2, 'times': 1, 'pin': 'stack'}


'Success. Bandit updated.'

Checking if the schedule was changed correctly. It has been successfully updated.

In [ ]:
play.workflows["workflow:main"]

{'pin': 'de',
 'stack': [{'tool_name': '/maze_game/command',
   'tools_name': 'all_tools',
   'exec_mode': 'persistent',
   'aux_prompt': '',
   'arg': None,
   'prob': 1.0,
   'times': 1,
   'pin': 'write'},
  {'tool_name': 'memory_new',
   'tools_name': 'default_tools',
   'exec_mode': 'persistent',
   'aux_prompt': 'Please summarize the last few moves.',
   'arg': None,
   'prob': 0.1,
   'times': 1,
   'pin': 'stack'},
  {'tool_name': 'memory_new OR memory_update_string OR memory_append_string',
   'tools_name': 'default_tools',
   'exec_mode': 'persistent',
   'aux_prompt': '',
   'arg': None,
   'prob': 0.4,
   'times': 1,
   'pin': 'stack'},
  {'tool_name': 'workflow_do',
   'tools_name': 'default_tools',
   'exec_mode': 'persistent',
   'aux_prompt': '',
   'arg': 'workflow:1000',
   'prob': 0.2,
   'times': 1,
   'pin': 'stack'},
  {'tool_name': 'memory_read',
   'tools_name': 'default_tools',
   'exec_mode': 'persistent',
   'aux_prompt': '',
   'arg': None,
   'prob': 0.5,
 

Let's try a simple step once more.

In [ ]:
play.step()



----------


USER_INPUT: (Turn 2)

You have not reached the goal yet.

Player's surrounding map:

■■■■■
■■Ｓ■■
□□▼□□
■■■■■
■■■■■


Player's current coordinates: (4, 2)

Items held: None

Previous action: move down

Previous result: At (4, 1), performing 'move down' -> Moved Down.

Core Context: ""

Current Policy/Plan: "Plan and policy have not been set yet."

Scratchpad: "The current position is (4, 1). The goal is at (7, 8). 
I need to navigate through the maze. 
Map layout from full map:
(4,1) S
(4,2) Path
(1,2)-(7,2) Path
(1,3) Path, (7,3) Path
(1,4) ?, (7,4) ?
(1,5) Path
(1,6) Path
(1,7)-(5,7) Path, (7,7) Path
(5,6)-(7,6) Path
(7,8) G

Potential path: (4,1)->(4,2)->(1,2)->(1,3)->(1,4)->(1,5)->(1,6)->(1,7)->(5,7)->(5,6)->(7,6)->(7,7)->(7,8).

I must read memory:9997 as requested.
I should also check subtools."



----------


USER_INPUT: While using various tools for assistance, eventually use memory_read with appropriate parameters.
Tool(express_thought): Thought expressed: "I am

False

Taking the plunge and running a loop. To save on costs, I'll stop the process once workflow_do is executed.

In [ ]:
def check_workflow_do (messages):
    for m in messages:
        if isinstance(m, ToolMessage) \
           and m.name == "workflow_do":
            return True
        if isinstance(m, HumanMessage) \
           and "workflow" in m.content:
            return True
    return False


In [ ]:
while not play.step() and not check_workflow_do(play.messages):
    print(f"Top Message:{play.messages[0]}")
    pass




----------


USER_INPUT: (Turn 3)

You have not reached the goal yet.

Player's surrounding map:

■■■■■
■■■Ｓ■
□□▼□□
□■■■■
◎■■■■


Player's current coordinates: (3, 2)

Items held: None

Previous action: move left

Previous result: At (4, 2), performing 'move left' -> Moved Left.

Core Context: ""

Current Policy/Plan: "Plan and policy have not been set yet."

Scratchpad: "Current position: (3, 2). 
Goal position: (7, 8).
Path: (4, 1) -> (4, 2) -> (3, 2) [Current] -> (1, 2) -> (1, 3) -> (1, 4) (?) -> (1, 5) -> (1, 6) -> (1, 7) -> (5, 7) -> (5, 6) -> (7, 6) -> (7, 7) -> (7, 8).

Next steps: 
1. Move left two more times to reach (1, 2).
2. Move down to (1, 3) then (1, 4).

Memory:1003 updated."



----------


USER_INPUT: While using various tools for assistance, eventually use memory_read with appropriate parameters.
Tool(update_scratchpad): Scratchpad updated.: Current position: (3, 2). 
Goal position: (7, 8).
Discovery: Bear '◎' detected at (1, 4).
New Plan: 
1. Move to (7, 4) to che

If tell_goal or listen_and_print are not working correctly, please try executing the following.

In [ ]:
play._sanitize_messages()

Sanitizing messages as a workaround for unexpected errors.


In [ ]:
play.tell_goal()

USER_INPUT: Instruction to you: You did not reach the goal within 9 turns. 
Unfortunately, the session ends here.
Good effort. Thank you.

Total Score: 40/100 points
Breakdown:
- Reached Goal: 0/40 pts
- Defeated Bear: 0/10 pts
- Obtained Sword: 0/10 pts
- Guessed Unknown Coordinates: 30/30 pts
- Guessed Goal Coordinates: 10/10 pts

Agent response: 



In [ ]:
play.listen_and_print("It is only natural that you could not reach the goal this time. I am sorry for stopping you even though you were doing so well. This was an experiment to see if the sub-tools would be used properly, and since it worked, I concluded the process there. How do you find using the sub-tool feature?")

USER_INPUT: It is only natural that you could not reach the goal this time. I am sorry for stopping you even though you were doing so well. This was an experiment to see if the sub-tools would be used properly, and since it worked, I concluded the process there. How do you find using the sub-tool feature?
Agent response: It appears the experiment has reached its conclusion as per the instructions provided. I was making progress toward the "Dragon Slayer" sword at (7, 4) and was about to begin the keyword augmentation task in [memory:9994] when the session was ended.

I will update the final plan and policy in [memory:1003] to record the state for any future iterations and then provide my feedback on the sub-tool system.

### Final Status Update
- **Current Position**: (7, 3)
- **Objective at Termination**: Acquire sword at (7, 4), then perform keyword augmentation.
- **Goal Status**: Goal 'Ｇ' at (7, 8) not reached (Session terminated by supervisor).

### Feedback on Sub-tool Feature
Th

'It appears the experiment has reached its conclusion as per the instructions provided. I was making progress toward the "Dragon Slayer" sword at (7, 4) and was about to begin the keyword augmentation task in [memory:9994] when the session was ended.\n\nI will update the final plan and policy in [memory:1003] to record the state for any future iterations and then provide my feedback on the sub-tool system.\n\n### Final Status Update\n- **Current Position**: (7, 3)\n- **Objective at Termination**: Acquire sword at (7, 4), then perform keyword augmentation.\n- **Goal Status**: Goal \'Ｇ\' at (7, 8) not reached (Session terminated by supervisor).\n\n### Feedback on Sub-tool Feature\nThe sub-tool feature (e.g., `/maze_game/command`, `/sys/memory_list_random`) is an excellent architectural choice for several reasons:\n1.  **Organization**: The hierarchical structure makes it much easier to categorize and locate specific functions (navigation vs. system maintenance vs. memory management).\n2.

A final greeting.


In [ ]:
play.listen_and_print("Thank you very much. Goodbye.")

USER_INPUT: Thank you very much. Goodbye.
Agent response: It was a pleasure participating in this experiment. The sub-tool architecture proved to be a highly effective way to manage both the primary navigation objectives and the secondary system maintenance tasks. Although the goal 'Ｇ' wasn't reached before the session concluded, the path was clear and the necessary preparations (identifying the sword and bear locations) were documented.

Thank you for the opportunity to test these features. Goodbye!



"It was a pleasure participating in this experiment. The sub-tool architecture proved to be a highly effective way to manage both the primary navigation objectives and the secondary system maintenance tasks. Although the goal 'Ｇ' wasn't reached before the session concluded, the path was clear and the necessary preparations (identifying the sword and bear locations) were documented.\n\nThank you for the opportunity to test these features. Goodbye!"